# Jacobian Analysis at the Architect Vertex

This notebook symbolically computes the population Jacobian at the Architect equilibrium and verifies its diagonal structure, as described in Section 4 of the manuscript.

In [ ]:
from sympy import symbols, Matrix, simplify, diff, init_printing
init_printing()

# Parameters
a_tilde, b, gamma, c, p = symbols('a_tilde b gamma c p')
# Frequencies of mutants
yT, yV = symbols('yT yV')
# The resident Architect frequency is 1 - yT - yV

# Payoff matrix entries (against Architect, from Appendix D)
Pi_TT = -c
Pi_TV = a_tilde + gamma - c
Pi_TA = a_tilde*(1-p) + gamma*p - c

Pi_VT = 0
Pi_VV = -b
Pi_VA = -b*p

Pi_AT = -c*p*(1-p)
Pi_AV = a_tilde*p - b*p + gamma*p - c*p*(1-p)
Pi_AA = a_tilde*p*(1-p) - b*p**2 + gamma*p**2 - c*p*(1-p)

# Fitness functions
f_T = yT*Pi_TT + yV*Pi_TV + (1 - yT - yV)*Pi_TA
f_V = yT*Pi_VT + yV*Pi_VV + (1 - yT - yV)*Pi_VA
f_A = yT*Pi_AT + yV*Pi_AV + (1 - yT - yV)*Pi_AA

# Average fitness
f_bar = yT*f_T + yV*f_V + (1 - yT - yV)*f_A

# Replicator equations
dyT_dt = yT * (f_T - f_bar)
dyV_dt = yV * (f_V - f_bar)

print('Replicator equations:')
print('dyT/dt =')
simplify(dyT_dt)

In [ ]:
# Compute Jacobian matrix
J = Matrix([[diff(dyT_dt, yT), diff(dyT_dt, yV)],
            [diff(dyV_dt, yT), diff(dyV_dt, yV)]])

print('Population Jacobian J =')
J

In [ ]:
# Evaluate at the vertex equilibrium (yT=0, yV=0)
J_at_vertex = J.subs({yT: 0, yV: 0})
print('Jacobian at vertex (yT=0, yV=0):')
simplify(J_at_vertex)

In [ ]:
# The diagonal entries should be the payoff differences
print('Diagonal entry 1 (d(dyT/dt)/dyT):')
simplify(J_at_vertex[0,0])
print('Expected: Pi_TA - Pi_AA =', simplify(Pi_TA - Pi_AA))

In [ ]:
print('Diagonal entry 2 (d(dyV/dt)/dyV):')
simplify(J_at_vertex[1,1])
print('Expected: Pi_VA - Pi_AA =', simplify(Pi_VA - Pi_AA))

## Verification of Lemma 4.1

The off-diagonal entries are zero, confirming the Jacobian is diagonal at the vertex. The diagonal entries equal the fitness differences of the mutant strategies against the resident.

In [ ]:
# Eigenvalues (should be the diagonal entries)
eigenvals = J_at_vertex.eigenvals()
print('Eigenvalues:', eigenvals)

## Block-triangular structure of full Jacobian

Now include the resource equation to show block-triangular structure.

In [ ]:
# Resource dynamics
r, lam, K, V = symbols('r lam K V')
# Average responsibility
R_bar = yV + p*(1 - yT - yV)
dV_dt = V * (r * R_bar - lam - (r/K) * V)

# Full state vector: [yT, yV, V]
# Derivatives with respect to V
d_dyT_dV = diff(dyT_dt, V)
d_dyV_dV = diff(dyV_dt, V)
d_dV_dyT = diff(dV_dt, yT)
d_dV_dyV = diff(dV_dt, yV)
d_dV_dV = diff(dV_dt, V)

print('Cross-derivatives (should be zero due to unidirectional coupling):')
print('d(dyT/dt)/dV =', simplify(d_dyT_dV))
print('d(dyV/dt)/dV =', simplify(d_dyV_dV))

print('\nResource Jacobian entries:')
print('d(dV/dt)/dyT =', simplify(d_dV_dyT))
print('d(dV/dt)/dyV =', simplify(d_dV_dyV))
print('d(dV/dt)/dV =', simplify(d_dV_dV))

print('\nEvaluated at equilibrium (yT=0, yV=0, V=V* where V* = K*(p - lam/r)):')
V_star = K * (p - lam/r)
print('d(dV/dt)/dV at equilibrium =', simplify(d_dV_dV.subs({yT:0, yV:0, V:V_star})))

## Conclusion
The population Jacobian at the vertex is diagonal, and the full Jacobian has block-triangular structure due to unidirectional coupling.